In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns', None)  
df = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')


In [ ]:
df.shape

In [ ]:
df.head(4)

In [ ]:
import seaborn as sns
sns.heatmap(df.isnull())

In [ ]:
df.corr()

In [ ]:
df['price_range'].value_counts()

0 = low price ,
1 = medium price,
2= high price,
3 = very high price


Let's see some data visualisation

# Let's see how camera mega pixels affect pricing 

Front Camera

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.countplot(df['fc'])


In [ ]:
sns.countplot(df['price_range'])

price Range and Front Camera MegaPixels have no significant relationship 

In [ ]:
sns.catplot(x='price_range',y='fc',kind='swarm',data=df)

In [ ]:
sns.catplot(x='price_range',y='fc',kind='box',data=df)

# Rear Camera Pixels and Price Category

In [ ]:
sns.countplot(df['pc'])

In [ ]:
sns.catplot(x='price_range',y='pc',kind='swarm',data=df)

In [ ]:
sns.catplot(x='price_range',y='pc',kind='box',data=df)

In [ ]:
sns.catplot(x='price_range',y='fc',kind='boxen',data=df)

In [ ]:
sns.catplot(x='price_range',y='fc',kind='point',data=df)

# RELATION BETWEEN BOTH TYPE OF CAMERAS AND PRICE TYPE

In [ ]:
plt.figure(figsize = (20,6))
sns.barplot(x="pc", y="fc", hue="price_range", data=df)
plt.xticks(rotation = 90)
plt.xlabel('primry camera pixles')
plt.ylabel('secondary camera pixels')
plt.show()

# RAM and PRICE RANGE

In [ ]:
plt.scatter(df['ram'],df['price_range'])
plt.xlabel('RAM(mb)')
plt.ylabel('price class')

> Let's see how range is distributed
most of the phones of lowest class have ram upto 2000 mb at most, medium phones have RAm atmost upto 3000mb and starting from 500 mb , higher category phones have RAM atleast 1500 mb and atmost 4000 mb , very high phones are starting from 2500mb and going more than 4000 mb 

In [ ]:
plt.bar(df['ram'],df['price_range']==0,label='low')
plt.bar(df['ram'],df['price_range']==1,color='red',label='medium')
plt.bar(df['ram'],df['price_range']==2, color='green',label='high')
plt.bar(df['ram'],df['price_range']==3,color='yellow',label='very high')
plt.legend()
plt.xlabel('RAM(mb)')
plt.ylabel('price class')

> it clearly shows that there is a direct relationship and a strong relationship among both

In [ ]:
plt.plot(df['ram'],df['price_range']==0,label='low')
plt.plot(df['ram'],df['price_range']==1,color='red',label='medium')
plt.plot(df['ram'],df['price_range']==2, color='green',label='high')
plt.plot(df['ram'],df['price_range']==3,color='yellow',label='very high')
plt.legend()
plt.xlabel('RAM(mb)')
plt.ylabel('price class')

In [ ]:
sns.catplot(x='price_range',y='ram',kind='swarm',data=df)

there are clear distinction between different classifictions

# FEATURE ENGINEERING AND FEATURE SELECTION

FIRST I WOULD CREATE A MODEL WITH ALL THE EXISTIONG COLUMNS THEN I WOULD MAKE OTHER MODELS AFTER APPLYING FEATURE SELECTION AND THEN CHECK RELATIVE PERFORMANCE OF MODEL

In [ ]:
x = df[['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi']]
y = df['price_range']

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train ,y_test = train_test_split(x,y,test_size=0.2,random_state=38)

# GRADIENT BOOST

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
boost = GradientBoostingClassifier()
boost.fit(x_train,y_train)
boost.score(x_test,y_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
y_pred = boost.predict(x_test)
cf_boost=confusion_matrix(y_test,y_pred)

In [ ]:

sns.heatmap(cf_boost, annot=True, fmt='', cmap='Blues')

In [ ]:
importance = boost.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
x.columns

In [ ]:
x_modified = df[['battery_power','clock_speed','fc','int_memory','m_dep','mobile_wt','n_cores','px_height','px_width','ram','sc_w','talk_time']]

In [ ]:
x_modified 

In [ ]:
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
x_modified = mm_scaler.fit_transform(x_modified)
x_modified

In [ ]:
x_train_new , x_test_new,y_train_new,y_test_new = train_test_split(x_modified,y,test_size=0.2,stratify = y,random_state=38)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
boost_mod = GradientBoostingClassifier()
boost_mod.fit(x_train_new,y_train_new)
boost_mod.score(x_test_new,y_test_new)

After Using Less Feature and Normalising Data , Test Accuracy increased by about 2 percent 

In [ ]:
importance = boost_mod.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance

Now Some Optimization For Our Algorithm : Gradient Boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
boost_mod = GradientBoostingClassifier(n_estimators=250,learning_rate=0.1,min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt')
boost_mod.fit(x_train_new,y_train_new)
boost_mod.score(x_test_new,y_test_new)